<a href="https://colab.research.google.com/github/KonstantinaT/Predictor/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries and functions

In [ ]:
import pandas as pd
!pip install Biopython
import Bio
import copy
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import gc
!pip install Biopython
import Bio
from Bio import SeqIO
from Bio.Seq import Seq

     |████████████████████████████████| 2.3 MB 4.3 MB/s 


In [ ]:
#Change the AA of the gene sequence 
def changeAAnew(original, target, position, sequence):
    #position = int(position)
    position -= 1 # because python indexation
    sequence = list(sequence)
    #print(len(sequence))
    #print(position)
    if sequence[position] == original:
        sequence[position]= target
        transcript = ''.join(sequence)
        return transcript
    else:
        tru = sequence[position]
        print('the original {}, in position {} not equal to: {}'.format(original, position+1, tru) )

### Cmc_mutations import

In [ ]:
cmc_all = pd.read_csv("/content/drive/MyDrive/my_colab/*2ndYear/cmc_export.csv",  delimiter = '\t') # COSMIC: cmc Tier 1,2,3, other mutations

# Filter missense / silent / nonsense mutations
missense_sil_nonsense_mut = cmc_all[(cmc_all['Mutation Description AA'] == 'Substitution - Missense') \
                              | (cmc_all['Mutation Description AA']== 'Substitution - coding silent') \
                                       | (cmc_all['Mutation Description AA']== 'Substitution - Nonsense')]

# creation of Tier1-2-3 mutations
Tier1_2_3 = missense_sil_nonsense_mut[(missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER'] == '1') \
                              | (missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER']== '2') \
                                       | (missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER']== '3')]
                             

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,24,49,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Read transcript consensus (glioma_All_cosmic.ipynb)
with open("/content/drive/MyDrive/my_colab/*2ndYear/transcript_sequences.csv", 'rb') as fp:
    gene_sequence = pickle.load(fp)

In [ ]:
# Keep mutations in position <= 2000
Tier1_2_3 = Tier1_2_3.reset_index(drop=True)
count_more_1 =0
position, AA_orig, AA_targ, sequence, indices_to_excl = [], [], [], [], []
for i, mut in Tier1_2_3.iterrows():

    string = mut['Mutation AA']
    if 'delins' in string:
      indices_to_excl.append(i)
      continue
    pos_temp = int(string[3:-1])
    if pos_temp > 2000:
      indices_to_excl.append(i)
      continue
    else:
      wt_sequence = str(gene_sequence[mut['ACCESSION_NUMBER']])
      if wt_sequence.count('*') > 1: # if there is a codon mid-protein
          count_more_1 +=1
      mut_seq = changeAAnew(string[2], string[-1], pos_temp,wt_sequence)
  
      if mut_seq is not None:
        if '*' in mut_seq:
          temp = mut_seq.split("*")
          mut_seq = temp[0]
        
        position.append(pos_temp)
        AA_orig.append(string[2])
        AA_targ.append(string[-1])

        sequence.append(mut_seq)
      else:
        indices_to_excl.append(i)

Tier1_2_3 = Tier1_2_3.drop(Tier1_2_3.index[indices_to_excl])
Tier1_2_3['position'] = position
Tier1_2_3['AA_orig'] = AA_orig
Tier1_2_3['AA_targ'] = AA_targ
Tier1_2_3['sequence'] = sequence

# Keep only relevant columns 
Tier1_2_3['label'] = [1]* len(Tier1_2_3)
Tier1_2_3 = Tier1_2_3[['GENE_NAME', 'ACCESSION_NUMBER', 'position', 'AA_orig', 'AA_targ', 'sequence', 'label']]

Tier_trancript_list = Tier1_2_3['ACCESSION_NUMBER'].unique()         

the original R, in position 95 not equal to: P
the original R, in position 95 not equal to: P
the original G, in position 248 not equal to: R
the original G, in position 74 not equal to: V


In [ ]:
# creation of benign mutations
cmcOther= missense_sil_nonsense_mut[(missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER'] == 'Other')]
# keep Transcripts with Pathogenic mutations
cmcOther = cmcOther[cmcOther['ACCESSION_NUMBER'].isin(Tier_trancript_list)]

In [ ]:
cmcOther = cmcOther.sample(frac=1).reset_index(drop=True) # shuffle and reset index
count = 0
count_synonymous = 0
position, AA_orig, AA_targ, sequence, indices_to_excl = [], [], [], [], []
for i, mut in cmcOther.iterrows():
  #if count <= 18000:
    string = mut['Mutation AA']
    if '=' in string:
      count_synonymous +=1
      indices_to_excl.append(i)
      continue
    if 'delins' in string:
      indices_to_excl.append(i)
      continue
    pos_temp = int(string[3:-1])
    if pos_temp > 2000:
      indices_to_excl.append(i)
      continue
    else:
      mut_seq = changeAAnew(string[2], string[-1], pos_temp, gene_sequence[mut['ACCESSION_NUMBER']])
        
      if mut_seq is not None:
        if '*' in mut_seq:
          temp = mut_seq.split("*")
          mut_seq = temp[0]
        
        position.append(pos_temp)
        AA_orig.append(string[2])
        AA_targ.append(string[-1])

        sequence.append(mut_seq)
        count+=1

      else:
        indices_to_excl.append(i)


cmcOther = cmcOther.drop(cmcOther.index[indices_to_excl])
cmcOther['position'] = position
cmcOther['AA_orig'] = AA_orig
cmcOther['AA_targ'] = AA_targ
cmcOther['sequence'] = sequence

# Keep only relevant columns 
cmcOther['label'] = [0]* len(cmcOther)
cmcOther = cmcOther[['GENE_NAME', 'ACCESSION_NUMBER', 'position', 'AA_orig', 'AA_targ', 'sequence', 'label']]

the original R, in position 197 not equal to: K
the original P, in position 172 not equal to: L
the original A, in position 1136 not equal to: G
the original T, in position 1851 not equal to: R
the original S, in position 305 not equal to: C
the original S, in position 205 not equal to: T
the original G, in position 1576 not equal to: V
the original P, in position 153 not equal to: R
the original L, in position 845 not equal to: V
the original Q, in position 88 not equal to: E
the original L, in position 417 not equal to: H
the original L, in position 27 not equal to: I
the original A, in position 336 not equal to: P


In [ ]:
count_synonymous # Synonymous mutations on COSMIC cmc_export dataset

127826

### Create balanced (protein) set

In [ ]:
# concatenate Tiers 1_2_3 with Other for the creation of balanced set
all_mutations = pd.concat([Tier1_2_3, cmcOther])
all_mutations = all_mutations.reset_index(drop=True) # shuffle and reset index

# label distribution per protein
lab_distr = {}
for ind, mut in all_mutations.iterrows():
  if mut['ACCESSION_NUMBER'] in lab_distr:
    if mut['label'] == 0:
      lab_distr[mut['ACCESSION_NUMBER']][0] +=1
    elif mut['label'] == 1:
      lab_distr[mut['ACCESSION_NUMBER']][1] +=1                  
  else:
    if mut['label'] == 0:
      lab_distr[mut['ACCESSION_NUMBER']] = [1, 0]
    elif mut['label'] == 1:
      lab_distr[mut['ACCESSION_NUMBER']] = [0, 1]

balanced_lab_distr = copy.deepcopy(lab_distr)
for i in balanced_lab_distr.keys():
  balanced_lab_distr[i][0] = 0

benign_to_excl = []
for i, mut in all_mutations.iterrows():

  if mut['label'] == 0:
    # have we reached the deleterious laber number of lab_distr?
    if balanced_lab_distr[mut['ACCESSION_NUMBER']][0] <= balanced_lab_distr[mut['ACCESSION_NUMBER']][1]:
      balanced_lab_distr[mut['ACCESSION_NUMBER']][0] +=1
    else:
      benign_to_excl.append(i)

all_mutations = all_mutations.drop(all_mutations.index[benign_to_excl])

In [ ]:
count_neg, count_pos = 0,0
for i, mut in all_mutations.iterrows():
  if mut['label'] == 0:
    count_neg +=1
  else:
    count_pos +=1
print(count_neg, count_pos)

18996 17639


In [ ]:
# save mutations
all_mutations.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_Other.csv', index=False)

In [ ]:
len(all_mutations['ACCESSION_NUMBER'].unique())

1361

In [ ]:
len(all_mutations)

36635

### For mmseq2_batch.ipynb (save Trascnrpt sequences to fasta files)

In [ ]:
# save Transcripts sequence into fasta files
alignment_path = '/content/drive/MyDrive/my_colab/3rdYear/datasets/alignments_Tier1_2_3_Other/'

for i in Tier_trancript_list:
  wt =str(gene_sequence[i])
  fasta_file = open(alignment_path + str(i) + '.fasta', 'w')
  fasta_file.write('>' + str(i) + "\n" + wt )
  fasta_file.close()


### TO DELETE - Clinvar - too few benign mutations

In [ ]:
# Keep ones with cllinvar benign annotations
# Import ClinVar dataset
path = "/content/drive/MyDrive/my_colab/*2ndYear/foldx/"
filename = "variant_summary0222.txt"

clinvar = pd.read_csv(path + filename, sep = '\t', usecols = ['Type', 'Name', 'GeneID', 'GeneSymbol', 'ClinicalSignificance', 'ClinSigSimple', 'PhenotypeList', 'OriginSimple', 'Assembly', 'ReviewStatus'])
clinvar= clinvar.loc[(clinvar['Type'] == 'single nucleotide variant') \
                      | (clinvar['Type'] == 'Variation') \
                      | (clinvar['Type'] == 'protein only')]

In [ ]:
clinvar_confirm = clinvar.loc[(clinvar['OriginSimple'] == 'somatic')  \
                      | (clinvar['OriginSimple'] == 'germline/somatic') \
                      | (clinvar['OriginSimple'] == 'not applicable') \
                      | (clinvar['OriginSimple'] == 'unknown') \
                      | (clinvar['OriginSimple'] == 'not provided') \
                      | (clinvar['OriginSimple'] == 'tested-inconclusive')]

In [ ]:
len(clinvar_confirm)

75289

In [ ]:
somatic_confirm = clinvar_confirm.loc[(clinvar_confirm['ReviewStatus'] == 'criteria provided, multiple submitters, no conflicts') \
                              | (clinvar_confirm['ReviewStatus'] == 'criteria provided, single submitter') \
                              | (clinvar_confirm['ReviewStatus'] == 'practice guideline') \
                              | (clinvar_confirm['ReviewStatus'] == 'reviewed by expert panel') ]

In [ ]:
print(len(somatic_confirm))

38650


In [ ]:
somatic_confirm.head()

,Type,Name,GeneID,GeneSymbol,ClinicalSignificance,ClinSigSimple,PhenotypeList,OriginSimple,Assembly,ReviewStatus
660,single nucleotide variant,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),4763,NF1,Pathogenic,1,"Neurofibromatosis, type 1|not provided|Heredit...",germline/somatic,GRCh37,"criteria provided, multiple submitters, no con..."
661,single nucleotide variant,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),4763,NF1,Pathogenic,1,"Neurofibromatosis, type 1|not provided|Heredit...",germline/somatic,GRCh38,"criteria provided, multiple submitters, no con..."
685,single nucleotide variant,NM_001042492.3(NF1):c.4021C>T (p.Gln1341Ter),4763,NF1,Pathogenic,1,"Neurofibromatosis, type 1|not provided",germline/somatic,GRCh37,"criteria provided, multiple submitters, no con..."
686,single nucleotide variant,NM_001042492.3(NF1):c.4021C>T (p.Gln1341Ter),4763,NF1,Pathogenic,1,"Neurofibromatosis, type 1|not provided",germline/somatic,GRCh38,"criteria provided, multiple submitters, no con..."
1390,single nucleotide variant,NM_024334.3(TMEM43):c.1073C>T (p.Ser358Leu),79188,TMEM43,Pathogenic,1,Arrhythmogenic right ventricular cardiomyopath...,germline/somatic,GRCh37,"criteria provided, multiple submitters, no con..."


In [ ]:
somatic_confirm_exon = somatic_confirm[somatic_confirm['Name'].str.contains(r'\+') == False]
somatic_confirm_exon = somatic_confirm_exon[somatic_confirm_exon['Name'].str.contains('-') == False]
somatic_confirm_exon = somatic_confirm_exon[somatic_confirm_exon['Name'].str.contains('p.') == True]
somatic_confirm_exon = somatic_confirm_exon[somatic_confirm_exon['Name'].str.contains('g') == False] # upstream variants
somatic_confirm_exon38 = somatic_confirm_exon[somatic_confirm_exon['Assembly'] == 'GRCh38']


print(len(somatic_confirm_exon38))

benign_clinvar = somatic_confirm_exon38.loc[(somatic_confirm_exon38['ClinicalSignificance'] == 'Benign/Likely benign') | \
                                        (somatic_confirm_exon38['ClinicalSignificance'] == 'Benign') | \
                                        (somatic_confirm_exon38['ClinicalSignificance'] == 'Likely benign') ] 

print(len(benign_clinvar))                                        

12076
833
